# HOG + SVC(kernel='linear')

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#!pip install opencv-python

In [3]:
import numpy as np
import scipy.linalg as sla
from PIL import Image
from scipy import misc
import scipy.sparse as sps
import scipy.sparse.linalg as spla
import timeit
%matplotlib inline

import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import pandas as pd

#from catboost import CatBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import LabelEncoder

from sklearn.decomposition import PCA

import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import plot_confusion_matrix

from matplotlib import cm

from tqdm.auto import tqdm

from google.colab import files

import zipfile # Библиотека для работы с zip архивами
import os  

from skimage import feature
import skimage.color as color

import glob
import random

import torch

from pandas.core.common import flatten

from torch.utils.data import Dataset, DataLoader

In [4]:
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

## Загрузка данных

In [5]:
zip = "/content/drive/MyDrive/dataset.zip"

In [6]:
# # Распаковываем архив
z = zipfile.ZipFile(zip, 'r')
z.extractall('/content/data_10k')

In [7]:
train_data_path = "/content/data_10k"

In [8]:
image_paths = [] # список с путями картинок
classes = [] # список классов


for data_path in glob.glob(train_data_path + '/*'):
    classes.append(data_path.split('/')[-1]) 
    image_paths.append(glob.glob(data_path + '/*'))
    
image_paths = list(flatten(image_paths))
random.shuffle(image_paths)

print('image_path example: ', image_paths[0])
print('class example: ', classes[0])

image_path example:  /content/data_10k/2/2_9838.png
class example:  8


In [9]:
len(image_paths)

80000

## Разобьем на train и test

In [10]:
x_train_path, x_test_path = train_test_split(image_paths, test_size=0.2, random_state=42)

In [11]:
len(x_test_path)

16000

## Класс загрузчика

In [12]:
class_to_idx = {'1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8}

In [13]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, image_paths):
        self.image_paths = image_paths

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_filepath = self.image_paths[idx]
        image = 255 - cv2.imread(image_filepath)

        hog_vec, hog_vis = feature.hog(image, visualize=True)
        
        label = image_filepath.split('/')[-2]
        label = class_to_idx[label]

        return hog_vis.ravel(), label

In [14]:
train_dataset = MyDataset(x_train_path)

test_dataset = MyDataset(x_test_path)

In [15]:
b_z = 100

num_work = 10

train_loader = DataLoader(train_dataset, batch_size=b_z, shuffle=False, num_workers=num_work)

test_loader = DataLoader(test_dataset, batch_size=b_z, shuffle=False, num_workers=num_work)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [16]:
#next(iter(train_loader))[1]

#Обучим модель

In [17]:
cl_all = SVC(kernel='linear')

i = 0

for imgs, labels in tqdm(train_loader):
  cl_all.partial_fit(imgs, labels, classes = np.array([1,2,3,4,5,6,7,8]))
 

  i = i + 1

  ac = []

  if (i % 10 == 0):
    for imgs_t, labels_t in tqdm(test_loader):

      pred_all = cl_all.predict(imgs_t)
      ac.append(accuracy_score(pred_all, labels_t))

    print(np.mean(ac))


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/640 [00:00<?, ?it/s]

AttributeError: ignored